In [2]:
import spacy
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from tqdm import notebook
from datetime import datetime
notebook.tqdm.pandas()

D:\Anaconda\lib\site-packages\tqdm\std.py:651: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
nlp2 = spacy.load('../Spacy models/kfold_3', disable=['parser', 'tagger', 'textcat'])
voc = pd.read_csv('vocop-clustered-new.csv', sep='	')

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (17,27,35,36,38) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## NER functions

In [9]:
def NER(text, nlp):
    ''' Takes in a string and uses the specified NLP model to tag entities within the string.
        Returns all entities tagged as PERSON within the string.
    '''
    # Tag text
    doc = nlp(text)
    holder = []
    
    # Append all entities tagged as PERSON
    for ent in doc.ents:
        if ent.label_ == 'PERSON' and ' ' in ent.text:
            holder.append(ent.text)
    return holder

## NEL functions

In [ ]:
def _float_feature(value):
        """Returns an float_list from a int/float."""
        return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def serialize_example(features):
    """
    Creates a tf.Example message ready to be written to a file.
    """
    
    # Create a dictionary mapping the feature name to the tf.Example-compatible data type.
    feature = {}
    for x in enumerate(features):
        feature[str(x[0] + 1)] = _float_feature(x[1])

    # Create a Features message using tf.train.Example.
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

def ranking(matches, directory):
    tags=["serve"]
    signature_def_key = "predict"
    saved_model_dir = directory
    holder = {}
    with Session() as sess:
        loader.load(sess, tags, saved_model_dir)
        serialized_examples = []
        for x in matches:
            vocop_ids = x
            names = matches[x][0]
            days = matches[x][1]
            locations = matches[x][2]
            ranks = matches[x][3]
            ships = matches[x][4]
            name_count = matches[x][5]
            serialized_example = serialize_example(names, days, locations, 
                                                   ranks, ships, name_count)
            serialized_examples.append(serialized_example)
        inputs_feed_dict = {'input_example_tensor:0': serialized_examples}
        outputs = sess.run('groupwise_dnn_v2/accumulate_scores/div_no_nan:0', feed_dict=inputs_feed_dict)
        output = [(outputs[x][0], vocop_ids[x]) for x in range(len(outputs))]
        holder[x] = output
    return holder

In [10]:
def find_matches(name, knowledgebase, target_column, distance=90):
    ''' Takes in a string containing the name of a person and returns all possible matches from the
        knowledgebase based on fuzzy string matching.
    '''
    matches = knowledgebase[(knowledgebase[target_column].astype(str).apply(fuzz.ratio, args=[name]) >= distance)]
    return matches

def convert_matches(entity, matches, name_dict):
    
    holder = {}
    notary_date = datetime.strptime(entity, '%Y-%m-%d')
    
    # Try to convert the dates for each entry into datetime format
    for x in range(matches.shape[0]):
        try:
            date1 = datetime.strptime(matches['date_begin_service_complete'].iloc[x], '%Y-%m-%d')
        except:
            date1 = datetime(year=1, month=1, day =1 )
        try:
            date2 = datetime.strptime(matches['date_end_service_complete'].iloc[x], '%Y-%m-%d')
        except:
            date2 = datetime(year=1, month=1, day =1 )
            
        # Keep only matches that are within distance days from notary_date
        if abs((notary_date - date1).days) < distance or abs((notary_date - date2).days) < distance:
            name_ratio = fuzz.ratio(entity, matches['fullNameOriginal'])
            name_count = name_dict[matches['fullNameNormalized']]
            vocid=matches['VOCOP_id']
            day_dif = min([abs((notary_date - date1).days), abs((notary_date - date2).days)])
            location = 
            rank =
            numships = 
            holder[vocid] = [name_ratio, day_dif, location, rank, numships, name_count]

    return holder

def NEL(entity, knowledgebase, model_dir):
    ''' Takes in a string containing an entity and returns either a match within the specified pandas 
        knowledgebase or None if no match is present.
    ''' 
    # Find and narrow down matches
    possible_matches = find_matches(entity, knowledgebase, 'fullNameOriginal')
    name_dict = knowledgebase.fullNameNormalized.value_counts()
    converted_matches = convert_matches(entity, possible_matches, name_dict)
    
    if converted_matches == []:
        return None
    ranked_matches = ranking(converted_matches, model_dir)
    return ranked_matches

In [16]:
x = NEL('Jan de Vrij', '1751-08-19', voc)

In [25]:
for x in {'lol': 12, 'sick':13}:
    print(x)

lol
sick
